# QSIPrep Basics: Diffusion MRI Preprocessing

This notebook demonstrates how to use the QSIPrep runner for diffusion MRI preprocessing.

## Overview

QSIPrep configures pipelines for processing diffusion-weighted MRI (dMRI) data. It performs:
- Head motion correction
- Susceptibility distortion correction
- Eddy current correction
- Spatial normalization
- Gradient direction correction

## Prerequisites

- Docker installed and running
- BIDS-formatted diffusion data
- FreeSurfer license file (required)
- Sufficient disk space for working directory

## Setup

In [1]:
from pathlib import Path
from voxelops import (
    run_qsiprep,
    QSIPrepInputs,
    QSIPrepDefaults,
)
import json

## Define Paths

In [2]:
# Input paths
bids_dir = Path("/media/storage/yalab-dev/qsiprep_test/heudiconv_test/")
participant = "01"
fs_license = Path("/home/galkepler/misc/freesurfer/license.txt")

# Output paths (optional)
output_dir = Path("/media/storage/yalab-dev/qsiprep_test/qsiprep_output/")
work_dir = Path("/media/storage/yalab-dev/qsiprep_test/work/qsiprep/")

## Basic Usage

### Option 1: Use Default Configuration

In [3]:
# Create inputs
inputs = QSIPrepInputs(
    bids_dir=bids_dir,
    participant=participant,
    output_dir=output_dir,
    work_dir=work_dir,
)

# Run with defaults (8 cores, 16GB RAM)
result = run_qsiprep(
    inputs,
    fs_license=fs_license,  # Required
)

print(f"Success: {result['success']}")
print(f"Duration: {result['duration_human']}")


Running qsiprep for participant 01
Command: docker run --rm -v /media/storage/yalab-dev/qsiprep_test/heudiconv_test:/data:ro -v /media/storage/yalab-dev/qsiprep_test/qsiprep_output:/out -v /media/storage/yalab-dev/qsiprep_test/work/qsiprep:/work -v /home/galkepler/misc/freesurfer/license.txt:/license.txt:ro pennlinc/qsiprep:1.0.2 /data /out participant --participant-label 01 --output-resolution 1.6 --nprocs 8 --mem-mb 16000 --work-dir /work --anatomical-template MNI152NLin2009cAsym --subject-anatomical-reference unbiased --fs-license-file /license.txt

Execution log saved: /media/storage/yalab-dev/qsiprep_test/logs/qsiprep_01_20260127_120018.json


ProcedureExecutionError: qsiprep failed: qsiprep failed with exit code 2

Stderr (last 1000 chars):
         [--b0-to-t1w-transform {Rigid,Affine}]
               [--intramodal-template-iters INTRAMODAL_TEMPLATE_ITERS]
               [--intramodal-template-transform {Rigid,Affine,BSplineSyN,SyN}]
               [--fs-license-file PATH]
               [--b0-motion-corr-to {iterative,first}]
               [--hmc-transform {Affine,Rigid}]
               [--hmc-model {none,3dSHORE,eddy,tensor}]
               [--eddy-config EDDY_CONFIG] [--shoreline-iters SHORELINE_ITERS]
               [--pepolar-method {TOPUP,DRBUDDI,TOPUP+DRBUDDI}]
               [--fmap-bspline] [--fmap-no-demean]
               [--use-syn-sdc [{warn,error}]] [--force-syn] [--version] [-v]
               [-w WORK_DIR] [--resource-monitor] [--config-file FILE]
               [--write-graph] [--stop-on-first-crash] [--notrack]
               [--debug {fieldmaps,pdb,all} [{fieldmaps,pdb,all} ...]]
               bids_dir

In [ ]:
raise

### Option 2: Override Resource Allocation

In [ ]:
# Run with more resources
result = run_qsiprep(
    inputs,
    fs_license=fs_license,
    nprocs=16,  # Use 16 cores
    mem_gb=32,  # Allocate 32GB RAM
)

print(f"Success: {result['success']}")
print(f"Used {result['config'].nprocs} cores and {result['config'].mem_gb}GB RAM")

### Option 3: Custom Configuration

In [ ]:
# Create custom brain bank configuration
config = QSIPrepDefaults(
    nprocs=24,
    mem_gb=64,
    output_resolution=1.5,  # 1.5mm isotropic
    output_spaces=["MNI152NLin2009cAsym", "T1w"],  # Multiple spaces
    longitudinal=True,  # Longitudinal processing
    skip_bids_validation=False,  # Always validate BIDS
    fs_license=fs_license,
    docker_image="pennlinc/qsiprep:1.0.2",  # Pin specific version
)

# Run with custom config
result = run_qsiprep(inputs, config)

print(f"Success: {result['success']}")

## Configuration Priority

Parameters can be specified in multiple ways with the following priority (highest to lowest):

1. **Keyword arguments** (highest priority)
2. **Environment variables** (if enabled)
3. **Config object**
4. **Defaults** (lowest priority)

In [ ]:
# Config says 16 cores, but keyword arg overrides to 32
config = QSIPrepDefaults(
    nprocs=16,
    mem_gb=32,
    fs_license=fs_license,
)

result = run_qsiprep(
    inputs,
    config,
    nprocs=32,  # This wins!
)

print(f"Actually used {result['config'].nprocs} cores")

## Inspect Execution Record

In [ ]:
# Execution metadata
print("Execution Details:")
print(f"  Tool: {result['tool']}")
print(f"  Participant: {result['participant']}")
print(f"  Start: {result['start_time']}")
print(f"  End: {result['end_time']}")
print(f"  Duration: {result['duration_human']} ({result['duration_seconds']:.1f}s)")
print(f"  Success: {result['success']}")
print(f"  Exit code: {result['exit_code']}")

# Configuration used
print("\nConfiguration:")
config_used = result['config']
print(f"  Cores: {config_used.nprocs}")
print(f"  Memory: {config_used.mem_gb}GB")
print(f"  Output resolution: {config_used.output_resolution}mm")
print(f"  Output spaces: {config_used.output_spaces}")
print(f"  Longitudinal: {config_used.longitudinal}")
print(f"  Docker image: {config_used.docker_image}")

## Check Expected Outputs

In [ ]:
outputs = result['expected_outputs']

print("Expected Output Locations:")
print(f"  QSIPrep directory: {outputs.qsiprep_dir}")
print(f"  Participant directory: {outputs.participant_dir}")
print(f"  HTML report: {outputs.html_report}")
print(f"  Figures directory: {outputs.figures_dir}")
print(f"  Work directory: {outputs.work_dir}")

# Verify outputs exist
print("\nOutput Validation:")
print(f"  HTML report exists: {outputs.html_report.exists()}")
print(f"  Participant dir exists: {outputs.participant_dir.exists()}")
print(f"  Figures dir exists: {outputs.figures_dir.exists()}")

## View HTML Report

QSIPrep generates a comprehensive HTML quality control report:

In [ ]:
from IPython.display import IFrame

# Display HTML report in notebook
if outputs.html_report.exists():
    IFrame(src=str(outputs.html_report), width=900, height=600)
else:
    print(f"HTML report not found at: {outputs.html_report}")

## List Output Files

In [ ]:
# List all preprocessed files
if outputs.participant_dir.exists():
    print(f"Files in {outputs.participant_dir}:\n")
    
    for f in sorted(outputs.participant_dir.rglob('*')):
        if f.is_file():
            rel_path = f.relative_to(outputs.participant_dir)
            size_mb = f.stat().st_size / (1024 * 1024)
            print(f"  {rel_path} ({size_mb:.1f} MB)")
else:
    print("Participant directory not found")

## Error Handling

In [ ]:
from voxelops.exceptions import (
    ProcedureExecutionError,
    InputValidationError,
    FreeSurferLicenseError,
)

try:
    result = run_qsiprep(
        inputs,
        fs_license=fs_license,
        nprocs=16,
    )
    print(f"Success: {result['success']}")
    
except InputValidationError as e:
    print(f"Input validation failed: {e}")
    print("  - Check that BIDS directory exists")
    print("  - Check that participant exists in BIDS directory")
    
except FreeSurferLicenseError as e:
    print(f"FreeSurfer license error: {e}")
    print("  - Obtain license from https://surfer.nmr.mgh.harvard.edu/registration.html")
    
except ProcedureExecutionError as e:
    print(f"Execution failed: {e}")
    print(f"  - Check log file: {result.get('log_file')}")
    print(f"  - Check stderr output")
    if 'stderr' in result:
        print(f"\nStderr (last 500 chars):\n{result['stderr'][-500:]}")
        
except Exception as e:
    print(f"Unexpected error: {e}")

## Batch Processing

In [ ]:
import time

# Get list of participants from BIDS directory
participant_dirs = sorted(bids_dir.glob('sub-*'))
participants = [d.name.replace('sub-', '') for d in participant_dirs if d.is_dir()]

print(f"Found {len(participants)} participants: {participants}\n")

# Process each participant
results = []
config = QSIPrepDefaults(
    nprocs=16,
    mem_gb=32,
    fs_license=fs_license,
)

for participant in participants:
    print(f"Processing participant {participant}...")
    
    inputs = QSIPrepInputs(
        bids_dir=bids_dir,
        participant=participant,
    )
    
    try:
        start = time.time()
        result = run_qsiprep(inputs, config)
        elapsed = time.time() - start
        
        results.append(result)
        print(f"  ✓ Success in {elapsed/60:.1f} minutes\n")
        
    except Exception as e:
        print(f"  ✗ Failed: {e}\n")
        results.append({
            "participant": participant,
            "success": False,
            "error": str(e),
        })

# Summary
successful = sum(1 for r in results if r.get('success'))
total_time = sum(r.get('duration_seconds', 0) for r in results if 'duration_seconds' in r)

print(f"\n{'='*60}")
print(f"Processed {len(results)} participants:")
print(f"  ✓ Successful: {successful}")
print(f"  ✗ Failed: {len(results) - successful}")
print(f"  Total time: {total_time/3600:.1f} hours")
print(f"  Average time: {total_time/len(results)/60:.1f} minutes per participant")

## Save Execution Records to Database

The execution records are plain Python dicts, making them easy to store in any database:

In [ ]:
# Example: Save to JSON file (for demonstration)
records_dir = Path("/data/records/qsiprep")
records_dir.mkdir(parents=True, exist_ok=True)

for result in results:
    if result.get('success'):
        participant = result['participant']
        record_file = records_dir / f"{participant}.json"
        
        with open(record_file, 'w') as f:
            json.dump(result, f, indent=2, default=str)
        
        print(f"Saved record for {participant} to {record_file}")

# In production, you would save to a proper database:
# db.save_processing_record(result)

## Reproduce a Previous Run

Since the exact Docker command is stored, you can reproduce any run:

In [ ]:
# Load a previous execution record
previous_record = json.loads((records_dir / "01.json").read_text())

# The exact command is stored
cmd = previous_record['command']
print("To reproduce this exact run, execute:")
print()
print(' '.join(cmd))

## Next Steps

After QSIPrep preprocessing:

1. Review the HTML QC report
2. Check the preprocessed images
3. Proceed to reconstruction with QSIRecon (see `03_qsirecon_basics.ipynb`)

## Tips

- **Resource allocation**: QSIPrep is memory-intensive. Allocate at least 16GB RAM.
- **Working directory**: Use fast local storage (SSD) for the work directory.
- **Longitudinal processing**: Enable for subjects with multiple sessions.
- **BIDS validation**: Always validate your BIDS dataset before running.
- **FreeSurfer license**: Required even if not running FreeSurfer reconstruction.
- **Output spaces**: Choose spaces based on downstream analysis needs.
- **Docker version**: Pin a specific image version for reproducibility.